# Animate WktGeom Map
## Notebook 6/7

## Gabriel del Valle
## 07/21/24
## NYC DATA SCIENCE ACADEMY

### For any questions about this project or to request full map videos or datasets, please feel free to reach out on Linkedin: 

   www.linkedin.com/in/gabriel-del-valle-147616152

   gabrielxdelvalle@gmail.com


In previous notebooks for animating maps I colored the streets on the map themselves for clarity while representing traffic data per datetime. However, this method requires aggregation, as there are multiple segements per street per datetime with their own unique rows. As a result, previous maps were a descriptive statistic rather than a form of simulation.

The maps produced by this notebook on the other hand are a much closer representation of the dataset, and at times comes close to simulating the elastic nature of traffic movement.

It does so by mapping the WktGeom data included in the original Automated Traffic Counts Dataset from OpenNYC, which indicates with coordinates the location at which data was recorded. When Plotted these coordinates create a circular marker but not a map. By combining these markers with the geojson map, I was able to give them context and a use.

---
 
Note that opposed to the previous mapping notebooks in this project, the data is not aggregated. Previously data was aggregated to account for multiple rows per street per datetime, representing multiple segments per street. Each data recording nodes accounts for these segments.

Nodes are colored with a congestion value, like in previous maps, however this is calculated with:

vol per street / max_vol per street

rather than:

avg_vol per street / max_vol per street


---

It takes hours to render the whole czone_October dataset, but on my macbook frames are produced almost once per second. 


## Functions:


### plot_geojson_wkt_geom( )

    Given a datetime, generate a Manhattan Congestion Relief Zone map which displays the data recording nodes as circles on the roads they monitor, using WktGeometry coordinates for accurate placement. Color the nodes with a congestion value indicating the level of street business.

### animate_map( )

    Given a start row and a number of rows to generate, loops through the datetime range of the congestion_streets dataset and inputs them into plot_geojson_wkt_geom()
    
### image_names()

    Given a start row and an end row corresponding to the congestion_streets dataset, image_names() produces a list of strings with the same date and index based naming schemes as plot_congestion_anim() and animate_bargraph_img().
    
    This is useful to quickly generate a list of specific files for operations in the video making process that require file names.
    
    
###  multiply_frames( )

    For the purpose of creating videos with the CV2 library, which has a minimum framerate of 24 frames per second. 

    In order to see each frame for half a second, multiply frames by 12

    Exports new multiplied frames to a new directory, divides names with a letter suffix

In [2]:
from shapely.wkt import loads
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
from shapely.geometry import MultiLineString, LineString
import re
import os
import cv2
from matplotlib.colors import Normalize
from matplotlib.cm import get_cmap
from matplotlib.colors import LinearSegmentedColormap

Import traffic data, specified to the Manhattan Congestion Relief Zone

In [3]:
czone_October = pd.read_csv("czone_October.csv")

Create a column for congestion, and format for use in mapping:

congestion = vol per street / max vol per street

In [4]:
# Recreate the congestion_streets dataframe with full columns
# And no aggregation
czone_October['datetime'] = pd.to_datetime(czone_October[['year', 'month', 'day', 'hour', 'minute']])

noNA_October = czone_October.dropna(subset=['Vol'])

maxStreetVol_Oct = noNA_October.groupby('street')['Vol'].max().reset_index()
maxStreetVol_Oct.columns = ['street', 'max_volume']
OTMD = pd.merge(noNA_October, maxStreetVol_Oct, on='street')

OTMD['congestion'] = OTMD['Vol'] / OTMD['max_volume']
OTMDdates = OTMD['datetime'].unique()

Import base map of Manhattan Congestion Relief Zone:

In [3]:
base_map = gpd.read_file('base_map.geojson')

Create a normalized color spectrum to represent congsetion value:

In [20]:
colors = ["green", "yellow", "red"]  # Simple green to yellow to red gradient
RGcmap = LinearSegmentedColormap.from_list("custom_green_red", colors, N=256)

Given a datetime, generate a Manhattan Congestion Relief Zone map which displays the data recording nodes as circles on the roads they monitor, using WktGeometry coordinates for accurate placement. Color the nodes with a congestion value indicating the level of street business.

Crucial to making geojson and WktGeom mapping systems work with each other is aligning their coordinate systems.

Outputs files to given directory

In [33]:

def plot_geojson_wkt_geom(datetime_value, output_dir, index):
    
    filtered_data = OTMD[OTMD['datetime'] == datetime_value]
    
    # Convert WKT strings in the 'WktGeom' column to Shapely geometry objects
    filtered_data['geometry'] = filtered_data['WktGeom'].apply(loads)

    # Create a GeoDataFrame from the filtered data
    wkt_gdf = gpd.GeoDataFrame(filtered_data, geometry='geometry')
    
    # Set the CRS for WKT geometries to the known original CRS, then convert to WGS84
    wkt_gdf.crs = "EPSG:2263"  # Assuming New York State Plane
    wkt_gdf = wkt_gdf.to_crs(epsg=4326)  # Convert to WGS84

    # Plotting
    fig, ax = plt.subplots(figsize=(10, 10))
    
    # Ensure that GeoJSON geometries are drawn below other plot elements
    #ax.set_axisbelow(True)
    
    # Plot the GeoJSON map, which should be in EPSG:4326
    base_map.plot(ax=ax, color='lightgray', edgecolor='none')
    
    # Overlay WKT geometries converted to the same CRS as json_streets
    # Use the 'congestion' column for coloring, applying the colormap
    wkt_gdf.plot(ax=ax, column='congestion', cmap=RGcmap, edgecolor='none', alpha=1.0, legend=False, zorder=2, markersize=10)

    # Remove axis borders and ticks
    ax.set_axis_off()

    
    # Optional: Adjust the viewing area to focus on a specific part of the map
    ax.set_xlim([-74.04, -73.94])
    ax.set_ylim([40.68, 40.79])
    
    ax.set_title(f'{datetime_value}')
    output_file = os.path.join(f'{index:04d}_{datetime_value}.png')
    output_path = f'{output_dir}/{output_file}'
    plt.savefig(output_path, bbox_inches='tight', pad_inches=0.1)
    plt.close()
    return(output_file)



Given a start row and a number of rows to generate, loops through the datetime range of the congestion_streets dataset and inputs them into plot_geojson_wkt_geom()

Assigns an output directory to plot_geojson_wkt_geom()


In [34]:
def animate_map(output_dir, start_row, num_rows):
    
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        
    file_names = [0] * num_rows
    end_row = start_row + num_rows

    # Iterate over rows of congave_October starting from start_row
    for i, datetime in enumerate(OTMDdates[start_row:end_row], start=start_row):
        # Check if we've reached the desired number of rows
        if i >= end_row:
            break
        
        # Call the plot_congestion function for the current datetime
        file_names[i-start_row] = plot_geojson_wkt_geom(datetime, output_dir, i)
        
    return file_names

image_names()

Given a start row and an end row corresponding to the congestion_streets dataset, image_names() produces a list of strings with the same date and index based naming schemes as plot_congestion_anim() and animate_bargraph_img().

This is useful to quickly generate a list of specific files for operations in the video making process that require file names.


In [6]:
def image_names(start_row, num_rows):
    file_names = [0] * num_rows
    end_row = start_row + num_rows
    for i, datetime in enumerate(OTMDdates[start_row:end_row], start=start_row):
        # Check if we've reached the desired number of rows
        if i >= end_row:
            break
        
        output_file = os.path.join(f'{i:04d}_{datetime}.png')
        file_names[i-start_row] = output_file

    return file_names

For the purpose of creating videos with the CV2 library, which has a minimum framerate of 24 frames per second. 

In order to see each frame for half a second, multiply frames by 12

Exports new multiplied frames to a new directory, divides names with a letter suffix

In [7]:
import shutil


alphabet = 'abcdefghijklmnopqrstuvwxyz'

def multiply_frames(factor, input_directory, output_directory, file_names):
    # Check if input directory exists
    if not os.path.exists(input_directory):
        raise FileNotFoundError(f"The directory {input_directory} does not exist.")
        
    num_files = len(file_names) * factor
    new_file_names = [0] * num_files
    index = 0
    
    # Create the output directory if it doesn't exist
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)
    
    
    # Process each file in the file_names list
    for title in file_names:
        file_path = os.path.join(input_directory, title)
        if not os.path.exists(file_path):
            continue  # Skip if file does not exist
        
        # Create duplicates
        for i in range(factor):
            suffix = alphabet[i % len(alphabet)]
            new_title = f"{title.rsplit('.', 1)[0]}_{suffix}.{title.rsplit('.', 1)[1]}"
            new_file_path = os.path.join(output_directory, new_title)
            shutil.copy(file_path, new_file_path)
            new_file_names[index] = new_title
            index +=1
    
    return new_file_names

In [8]:
import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")
# Re-enable warnings
#warnings.filterwarnings("default")

Generate maps for each unique datetime in the dataset:

In [9]:
len(OTMDdates)

10716

Map frames are generated 1000 frames at a time to avoid crashing the python Kernel. You may need to adjust based on your machine.

In [35]:
wktgeom_Mapbatch01 = animate_map("wktgeom_Map", 0, 1000)

In [37]:
wktgeom_Mapbatch02 = animate_map("wktgeom_Map", 1000, 1000)

In [38]:
wktgeom_Mapbatch03 = animate_map("wktgeom_Map", 2000, 1000)

In [39]:
wktgeom_Mapbatch04 = animate_map("wktgeom_Map", 3000, 1000)

In [40]:
wktgeom_Mapbatch05 = animate_map("wktgeom_Map01", 4000, 1000)

In [41]:
wktgeom_Mapbatch06 = animate_map("wktgeom_Map01", 5000, 1000)

In [42]:
wktgeom_Mapbatch07 = animate_map("wktgeom_Map01", 6000, 1000)

In [43]:
wktgeom_Mapbatch08 = animate_map("wktgeom_Map01", 7000, 1000)

In [44]:
wktgeom_Mapbatch09 = animate_map("wktgeom_Map01", 8000, 1000)

In [45]:
wktgeom_Mapbatch10 = animate_map("wktgeom_Map01", 9000, 1000)

In [46]:
wktgeom_Mapbatch11 = animate_map("wktgeom_Map01", 10000, 716)

Create a single list of image names to use with wktgeom_Map_12frame( )

In [47]:
wktgeom_Mapbatch_names = image_names(0, 10716)

Multiply frames to ensure each map is displayed for 0.5 seconds in the video. 

Adjust number of frames to change duration.

In [48]:
wktgeom_Map_12frame = multiply_frames(12, "wktgeom_Map", "wktgeom_Map_12frame", wktgeom_Mapbatch_names)

Convert frames to video:

In [49]:
fourcc = cv2.VideoWriter_fourcc(*'mp4v')

# Open the first image to get its dimensions
first_image_path = f'wktgeom_Map_12frame/{wktgeom_Map_12frame[0]}'
first_image = cv2.imread(first_image_path)
frame_width = first_image.shape[1]
frame_height = first_image.shape[0]


#aspect_ratio = frame_width / frame_height

# Initialize the VideoWriter object with the calculated dimensions
output_video = cv2.VideoWriter('wktgeom_Map.mp4', fourcc, 24, (frame_width, frame_height))

# Iterate over each image and add it to the video
for image in wktgeom_Map_12frame:
    image_path = f'wktgeom_Map_12frame/{image}'
    img = cv2.imread(image_path)
    output_video.write(img)

# Release the VideoWriter object
output_video.release()

[ WARN:0@170715.369] global loadsave.cpp:248 findDecoder imread_('wktgeom_Map_12frame/0'): can't open/read file: check file path/integrity
[ WARN:0@170715.369] global loadsave.cpp:248 findDecoder imread_('wktgeom_Map_12frame/0'): can't open/read file: check file path/integrity
[ WARN:0@170715.369] global loadsave.cpp:248 findDecoder imread_('wktgeom_Map_12frame/0'): can't open/read file: check file path/integrity
[ WARN:0@170715.369] global loadsave.cpp:248 findDecoder imread_('wktgeom_Map_12frame/0'): can't open/read file: check file path/integrity
[ WARN:0@170715.370] global loadsave.cpp:248 findDecoder imread_('wktgeom_Map_12frame/0'): can't open/read file: check file path/integrity
[ WARN:0@170715.370] global loadsave.cpp:248 findDecoder imread_('wktgeom_Map_12frame/0'): can't open/read file: check file path/integrity
[ WARN:0@170715.370] global loadsave.cpp:248 findDecoder imread_('wktgeom_Map_12frame/0'): can't open/read file: check file path/integrity
[ WARN:0@170715.370] global

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

